# Addressing the Problem

### Skin Disease Classification

The U.S. Census Bureau projects that by 2050, about half of all patients seen in the U.S. healthcare system will have skin of color (SOC). As these communities continue to grow, so will the likelihood of dermatologists encountering cutaneous (skin) diseases that happen more frequently in SOC patients, happen exclusively in SOC patients, and/or present differently in SOC patients than their White counterparts. Aiding dermatologists in their understanding, and more importantly diagnoses, of SOC patients’ cutaneous disease presentations is paramount to delivering life-saving
quality of care to these communities. As a result, the following practicum project will explore the use of computer vision for classification of cutaneous diseases in SOC patients.

### Problem Structure

The following research notebook addresses the supervised classification problem of determining whether a given image of a skin disease is benign (0) or malignant (1). The metrics to be used for evaluating the neural network's performance are binary cross-entropy and accuracy.

### Import Necessary Modules

In [1]:
# Python libraries
import shutil
import os
from IPython.display import display
from PIL import Image

# Data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Neural network libraries
import tensorflow as tf
from keras import utils, preprocessing, layers, optimizers, callbacks

# AWS libraries
import boto3

2024-04-17 16:08:59.646229: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Retrieval

### Data Fetch

My dataset of choice is the Diverse Dermatology Images (DDI) dataset curated by the Stanford University School of Medicine. As can be seen below, we begin by fetching the metadata CSV from the appropriate S3 location. The data workflow pipeline has already run the metadata transform Glue job on the raw metadata prior to the execution of this notebook, so we're merely fetching the cleaned product. However, further feature engineering will be performed on the metadata and images in preparation for model fitting.

In [2]:
def download_s3_images(images_file_key):  
    for image_file_name in ddi_df['DDI_file'].tolist():    
        output_path = 'images/{}'.format(image_file_name)
        
        with open(output_path, 'wb') as image_output:
            s3.download_fileobj(
                Bucket=bucket, 
                Key='{}/{}'.format(images_file_key, image_file_name),
                Fileobj=image_output
            )

In [3]:
s3 = boto3.client('s3')
bucket = 'poc-skin-disease-detection-cv'

In [4]:
data_file_key = 'ddidiversedermatologyimages/metadata/transform'
data_file_name = 'part-00000-f5ce70f1-c572-4cbc-818e-e8dda587c3ef-c000.csv'

obj = s3.get_object(Bucket=bucket, Key='{}/{}'.format(data_file_key, data_file_name)) 
ddi_df = pd.read_csv(obj['Body'])

In [5]:
ddi_df.head()

,DDI_file,skin_tone,malignant,disease
0,000001.png,56,1,melanoma-in-situ
1,000002.png,56,1,melanoma-in-situ
2,000003.png,56,1,mycosis-fungoides
3,000004.png,56,1,squamous-cell-carcinoma-in-situ
4,000005.png,12,1,basal-cell-carcinoma


In [6]:
if not os.path.exists('images/'):
    os.mkdir('images')
    download_s3_images(images_file_key='ddidiversedermatologyimages/images')

### Data Inspection

Upon further inspection of the metadata, we can also see that it contains 656 fully-populated records, two of which being integer features and the other two being string features.

In [7]:
ddi_df.shape

(656, 4)

In [8]:
ddi_df.dtypes

DDI_file     object
skin_tone     int64
malignant     int64
disease      object
dtype: object

In [9]:
ddi_df.isnull().sum()

DDI_file     0
skin_tone    0
malignant    0
disease      0
dtype: int64

In [10]:
ddi_df.describe()

,skin_tone,malignant
count,656.000000,656.000000
mean,33.966463,0.260671
std,17.511578,0.439336
min,12.000000,0.000000
25%,12.000000,0.000000
50%,34.000000,0.000000
75%,56.000000,1.000000
max,56.000000,1.000000


# Exploratory Data Analysis (EDA)

### Which cutaneous diseases comprise our dataset?

The initial scope of this project was to specifically assess melanomas. However, a total of 78 cutaneous disease presentations, including other well-known cutaneous conditions such as carcinomas, have been aggregated to form the DDI dataset.

In [11]:
print("Number of disease presentations: ", len(ddi_df['disease'].unique()))
np.sort(ddi_df['disease'].unique())

Number of disease presentations:  78


array(['abrasions-ulcerations-and-physical-injuries', 'abscess',
       'acne-cystic', 'acquired-digital-fibrokeratoma',
       'acral-melanotic-macule', 'acrochordon', 'actinic-keratosis',
       'angioleiomyoma', 'angioma', 'arteriovenous-hemangioma',
       'atypical-spindle-cell-nevus-of-reed', 'basal-cell-carcinoma',
       'basal-cell-carcinoma-nodular', 'basal-cell-carcinoma-superficial',
       'benign-keratosis', 'blastic-plasmacytoid-dendritic-cell-neoplasm',
       'blue-nevus', 'cellular-neurothekeoma', 'chondroid-syringoma',
       'clear-cell-acanthoma', 'coccidioidomycosis',
       'condyloma-accuminatum', 'dermatofibroma', 'dermatomyositis',
       'dysplastic-nevus', 'eccrine-poroma',
       'eczema-spongiotic-dermatitis', 'epidermal-cyst',
       'epidermal-nevus', 'fibrous-papule', 'focal-acral-hyperkeratosis',
       'folliculitis', 'foreign-body-granuloma', 'glomangioma',
       'graft-vs-host-disease', 'hematoma', 'hyperpigmentation',
       'inverted-follicular-k

### Which skin tones are present in this dataset?

What makes the DDI dataset unique is its inclusion of skin types across the skin tone spectrum. The skin tone index values below are representations of the Fitzpatrick Skin Types (FST): fairer skin tones (Types 1-2), medium skin tones (Types 3-4), and darker skin tones (Types 5-6). Given the record counts associated with each FST, we can conclude that the dataset samples are relatively balanced.

In [12]:
ddi_df.drop(['DDI_file', 'malignant'], axis=1).groupby(['skin_tone']).count()

,disease
skin_tone,
12,208
34,241
56,207


### What is the dataset's target variable distribution?

As was mentioned in the opening section, a skin disease presentation can be classified as either benign (0) or malignant (1); this is our dataset's target variable. Based on the data presented below, we can see that approximately 1 in 4 (26.1%) presentations will be malignant, a probability we should keep in mind when evaluating the classifications generated by the selected neural network. Additionally, we can see the classes are relatively balanced in their distribution across the FSTs, so no data resampling should be necessary.

In [13]:
pd.concat([ddi_df['malignant'].value_counts(), ddi_df['malignant'].value_counts(normalize=True)], axis=1)

,count,proportion
malignant,,
0,485,0.739329
1,171,0.260671


In [14]:
ddi_df.drop('DDI_file', axis=1).groupby(['malignant', 'skin_tone']).count()

disease
malignant skin_tone         
0         12             159
          34             167
          56             159
1         12              49
          34              74
          56              48

# Feature Engineering

### Data Augmentation

Data augmentation is a process by which new data is artificially generated based upon existing data, thereby enhancing the size of the original dataset. You may recall from the Data Retrieval section that only 656 records exist in the original DDI dataset, and that simply isn't enough data to effectively train the neural network.

3 augmentations were created for each image in the DDI dataset: 90-degree rotation, 180-degree rotation, and 270-degree rotation. Prefixes were prepended to the corresponding image filenames and added as "new" records to the metadata dataframe, giving us a new total of 2,624 images & records. At this stage in the feature engineering, all images belong to the images/ folder.

In [15]:
def augment_images(df, num_augs):
      for i in range(num_augs):
        aug_iter = i+1
        
        for filename in df['DDI_file']:
            img = Image.open('images/{}'.format(filename)).rotate(90*aug_iter, expand=True)
            img.save('images/rand{}_{}'.format(aug_iter, filename))

In [16]:
def augment_metadata(df, num_augs):
    orig_size = len(df)
    
    for i in range(num_augs):
        aug_iter = i+1
        
        df_dupe = df.iloc[:orig_size]
        df_dupe['DDI_file'] = df_dupe['DDI_file'].apply(lambda x: 'rand{}_{}'.format(aug_iter, x))
        df = pd.concat([df, df_dupe])
        df.index = range(len(df))

    return df

In [17]:
num_augs = 3

if len(os.listdir('images/')) == len(ddi_df):
    augment_images(ddi_df, num_augs)

ddi_df = augment_metadata(ddi_df, num_augs)

/tmp/ipykernel_2656/2880849854.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dupe['DDI_file'] = df_dupe['DDI_file'].apply(lambda x: 'rand{}_{}'.format(aug_iter, x))


### Creating Dataset Partitions

Once augmentation is completed, we have to separate all the image-metadata pairs into training, validation, and test data splits for model fitting and evaluation. 

We begin by splitting the metadata dataframes first using scikit-learn's train_test_split function, which permits stratified splits; this allows for each data split to have the same target variable and skin tone distributions. We then use the filenames associated with each data split to segregate the images into train/, val/, and test/ subfolders of the images/ folder. Each subfolder is then read into a keras ImageDataGenerator to link the images to their corresponding metadata data split. Finally, the ImageDataGenenator object is converted to a tensorflow dataset for improved model fitting performance.

It's also important to note the global variable declarations in this section. The target image size (img_height, img_width) used is 256x256, which is a recommendation I found in online research. The overwhelming majority of images in our dataset have larger dimensions than the target size, requiring image interpolation (resizing). The Lanczos algorithm was selected for interpolation based on online research into best methods for downsizing images.

In [18]:
def generate_dataset(df, data_split, shuffle=True):
    idg_data = preprocessing.image.ImageDataGenerator().flow_from_dataframe(
        df,
        directory='images/{}/'.format(data_split),
        x_col='DDI_file',
        y_col='malignant',
        target_size=(img_height, img_width),
        class_mode='binary',
        batch_size=batch_size,
        shuffle=shuffle,
        seed=42,
        interpolation='lanczos',
    )

    return tf.data.Dataset.from_generator(
        generator=lambda: idg_data,
        output_signature=(
            tf.TensorSpec(shape=(None, img_height, img_width, img_channels), dtype=tf.float32, name=None),
            tf.TensorSpec(shape=(None,), dtype=tf.float32, name=None)
        )
    ).apply(
        tf.data.experimental.assert_cardinality(np.ceil(len(df) / batch_size))
    )

In [19]:
X = ddi_df.drop('malignant', axis=1)
y = ddi_df['malignant']

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=0.7, random_state=42,
                                                            stratify=pd.concat([y, X['skin_tone']], axis=1))

X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, train_size=0.5, random_state=42,
                                                stratify=pd.concat([y_val_test, X_val_test['skin_tone']], axis=1))

In [20]:
data_splits = {
    'train': X_train,
    'val': X_val,
    'test': X_test,
}
label_splits = {
    'train': y_train,
    'val': y_val,
    'test': y_test,
}

In [21]:
for split in data_splits.keys():
    if not os.path.exists('images/{}/'.format(split)):
        os.mkdir('images/{}'.format(split))
    
        for filename in data_splits[split]['DDI_file']:
            shutil.move('images/{}'.format(filename), 'images/{}/{}'.format(split, filename))

In [22]:
batch_size = 32
img_height, img_width, img_channels = 256, 256, 3

In [23]:
ddi_df_train = pd.concat([X_train, y_train.astype(str)], axis=1)
images_train_ds = generate_dataset(ddi_df_train, 'train')

Found 1836 validated image filenames belonging to 2 classes.


In [24]:
ddi_df_val = pd.concat([X_val, y_val.astype(str)], axis=1)
images_val_ds = generate_dataset(ddi_df_val, 'val')

Found 394 validated image filenames belonging to 2 classes.


In [25]:
X_test.sort_index(inplace=True)
y_test.sort_index(inplace=True)

ddi_df_test = pd.concat([X_test, y_test.astype(str)], axis=1)
images_test_ds = generate_dataset(ddi_df_test, 'test', shuffle=False)

Found 394 validated image filenames belonging to 2 classes.


# Model Architecture

In [26]:
model = tf.keras.Sequential([
    tf.keras.Input((img_height, img_width, img_channels)),
    layers.Rescaling(1./255),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics='accuracy',
)

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 127, 127, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0

# Model Evaluation

In [28]:
model.fit(
    x=images_train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE),
    validation_data=images_val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE),
    validation_steps=len(images_val_ds),
    epochs=10,
    callbacks=callbacks.EarlyStopping(patience=2, restore_best_weights=True, start_from_epoch=3, verbose=1),
)

Epoch 1/10


2024-04-17 16:09:02.279069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - ETA: 0s - loss: 0.8093 - accuracy: 0.5376

2024-04-17 16:09:56.834977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 64s 1s/step - loss: 0.8093 - accuracy: 0.5376 - val_loss: 0.6235 - val_accuracy: 0.7386
Epoch 2/10
58/58 [==============================] - 40s 683ms/step - loss: 0.6149 - accuracy: 0.6601 - val_loss: 0.5964 - val_accuracy: 0.7386
Epoch 3/10
58/58 [==============================] - 40s 682ms/step - loss: 0.5096 - accuracy: 0.7603 - val_loss: 0.5800 - val_accuracy: 0.7411
Epoch 4/10
58/58 [==============================] - 40s 683ms/step - loss: 0.4319 - accuracy: 0.8344 - val_loss: 0.5732 - val_accuracy: 0.7386
Epoch 5/10
58/58 [==============================] - 40s 683ms/step - loss: 0.3709 - accuracy: 0.8965 - val_loss: 0.5801 - val_accuracy: 0.7360
Epoch 6/10
58/58 [==============================] - 40s 684ms/step - loss: 0.3213 - accuracy: 0.9390 - val_loss: 0.5922 - val_accuracy: 0.7234
Epoch 6: early stopping


In [29]:
y_preds = [1 if pred > 0.5 else 0 for pred in model.predict(images_test_ds)]

2024-04-17 16:13:24.618080: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


13/13 [==============================] - 9s 692ms/step


In [30]:
np.round(accuracy_score(y_test, y_preds), 3)

0.734

In [31]:
eval_df = X_test.drop('DDI_file', axis=1)
eval_df['label'] = y_test
eval_df['pred'] = y_preds
eval_df['correct'] = (eval_df['pred'] == eval_df['label']).astype(int)

In [32]:
np.round(eval_df[['skin_tone', 'correct']].groupby('skin_tone').mean(), 3)

,correct
skin_tone,
12,0.762
34,0.681
56,0.766


In [33]:
np.round(eval_df[['label', 'correct']].groupby('label').mean(), 3)

,correct
label,
0,0.993
1,0.000
